In [1]:
import numpy as np
import torch
from __future__ import print_function
from torch import nn
from torch import autograd
import torch.optim as optim
import torch.nn.functional as F
import math
from torch.autograd import Variable
import copy
import pandas
import Queue
import pyedflib

In [2]:
import pyedflib
import numpy as np
f = pyedflib.EdfReader("edf1")
n = f.signals_in_file
signal_labels = f.getSignalLabels()
sigbufs = np.zeros((n, f.getNSamples()[0]))
for i in np.arange(n):
    sigbufs[i, :] = f.readSignal(i)

In [3]:
time = np.array([201,1401,2601,3801,5001,6201,7401,8601,9801,11001,12201,13401,14601,15801,17001,18201,19401,20601,21801,23001,24201,25401,26601,27801,29001,30201, 31401])


In [4]:
from os import walk
files = []
dirs = ["happy", "loss", "neut"]
q_data_set = Queue.Queue()
q_targets = Queue.Queue()
for k in np.arange(3):
    files = []
    for (dirpath, dirnames, filenames) in walk(dirs[k]):
        files.extend(filenames)
        break
    for instance, file  in enumerate(files):
        f_read = pyedflib.EdfReader(dirs[k] +"/"+ file)
        n = f_read.signals_in_file
        signal_labels = f_read.getSignalLabels()
        sigbufs = np.zeros((n, f_read.getNSamples()[0]))
        shape = f_read.readSignal(0).shape[0]
        data =  np.zeros((n-1, shape))
        for i in np.arange(n-1):
            data[i, :] = f_read.readSignal(i)
        i = 0
        while (i < 26 and time[i] < shape):
            q_data_set.put(copy.deepcopy(data[:, time[i]:time[i+1]]))
            q_targets.put(k)
            i = i+1

In [5]:
i = 0
while (not q_data_set.empty()):
    temp = q_data_set.get_nowait()
    if (temp.shape[0] != 62 or temp.shape[0] != 1200):
        print (temp.shape[0])
        i += 1

62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
62
6

In [6]:
i

2296

In [5]:
data_set = np.zeros((q_data_set.qsize() - 69, 62, 1200))
targets = np.zeros((q_targets.qsize() - 69))

i = 0
j = 0
while (not q_data_set.empty()):
    temp = q_data_set.get_nowait()
    temp_targ = q_targets.get_nowait()
    if (temp.shape[1] != 1200 ):
        continue
    
    data_set[j] = copy.deepcopy(temp[:62, :])
    targets[j] = copy.deepcopy(temp_targ)
    j += 1

In [7]:
data_set.shape
targets.shape

(2227,)

In [69]:
sample = sigbufs[:, 201:1401]
sample = np.expand_dims(sample, axis = 0)
sample = np.vstack((sample, sample))

In [10]:
class LikeDislikeModel(nn.Module):
    def __init__(self, 
                 batch_size,
                 in_channels,
                 out_size,
                 L_sizes = [300, 200, 300],
                 fc_size = 20,
                 n_local_pred = 3,
                 f_sizes = [3, 3, 5], 
                 channels = [24, 16, 8], 
                 strides = [2, 2, 3], 
                 use_cuda = True):
        
        super(LikeDislikeModel, self).__init__()
        
        self.in_channels = in_channels
        self.batch_size = batch_size
        self.fc_size = fc_size
        self.out_size = out_size
        self.n_local_pred = n_local_pred
        self.use_cuda = use_cuda
        self.L_sizes = L_sizes
        
        self.f_sizes = f_sizes
        self.channels = channels
        self.strides = strides
        self.len_sizes = self._len_sizes()
        
        self.conv1 = nn.ModuleList()
        self.conv2 = nn.ModuleList()
        self.conv3 = nn.ModuleList()
        self.fc1 = nn.ModuleList()
        self.fc2 = nn.ModuleList()
        
        # TODO : add batchnormalization for each conv 
        self.batchnorm = nn.ModuleList()
        
        
        
        
        for i in range (n_local_pred):
            
            self.conv1.append(nn.Conv1d(in_channels, channels[0], f_sizes[0], strides[0]))
            self.conv2.append(nn.Conv1d(channels[0], channels[1], f_sizes[1], strides[1]))               
            self.conv3.append(nn.Conv1d(channels[1], channels[2], f_sizes[2], strides[2]))
            
            self.fc1.append(nn.Linear(self.len_sizes[i][-1] * self.channels[-1], fc_size))
            self.fc2.append(nn.Linear(fc_size, out_size))
            self.batchnorm.append(nn.ModuleList())
            for convNum in range(3):
                self.batchnorm[i].append(nn.BatchNorm1d(channels[convNum]))
            
        self.dropout = nn.Dropout(p=0.5)
        
    def _len_sizes(self):
        len_sizes = []
        for i in range (self.n_local_pred):
            temp = []
            for j in range (len(self.L_sizes)):
                if j == 0:
                    temp.append(int(math.floor((self.L_sizes[i] - self.f_sizes[j])/self.strides[j] + 1)))
                else:
                    temp.append(int(math.floor((temp[j-1] - self.f_sizes[j])/self.strides[j] + 1)))
            len_sizes.append(copy.deepcopy(temp))
        return len_sizes

    def preprocessing(self, input_batch):
        nns = []
        nns.append(input_batch[:, :, 200:500])
        nns.append(input_batch[:, :, 500:700])
        nns.append(input_batch[:, :, 700:1000])
        for i in range (self.n_local_pred):
            zero_mean = nns[i] - nns[i].mean()
            for n in range(input_batch.shape[0]):
                nns[i][n] = zero_mean[n, :, :]/(np.std(zero_mean[n, :, :],axis = 0))
                
            if self.use_cuda:
                nns[i] = Variable(torch.FloatTensor(copy.deepcopy(nns[i]))).cuda()
            else:
                nns[i] = Variable(torch.FloatTensor(copy.deepcopy(nns[i])))
                

        return nns

    def createLocalNetworks(self, nns):
        raw_output = []
        for nn_num in range( self.n_local_pred):
            conv1 = self.conv1[nn_num](nns[nn_num])
            bn1 = self.batchnorm[nn_num][0](conv1)
            relu1 = F.relu(bn1)

            conv2 = self.conv2[nn_num](relu1)
            bn2 = self.batchnorm[nn_num][1](conv2)
            relu2 = F.relu(bn2)
            
            conv3 = self.conv3[nn_num](relu2)
            bn3 = self.batchnorm[nn_num][2](conv3)
            relu3 = F.relu(bn3)
            
            relu3 = relu3.view(self.batch_size, -1)
            
            fc1 = self.fc1[nn_num](relu3)
            relu4 = F.relu(fc1)

            do = self.dropout(relu4)

            fc2 = self.fc2[nn_num](do)
            
            raw_output.append(F.log_softmax(fc2))
        return raw_output
    
    def _loss(self, raw_output, targets):
        
        if self.use_cuda:
            scores = Variable(torch.FloatTensor(self.batch_size, self.out_size).zero_()).cuda()
        else:
            scores = Variable(torch.FloatTensor(self.batch_size, self.out_size).zero_())
            
    
        for nn_num in range( self.n_local_pred):
            scores = scores + raw_output[nn_num]
        
        
        if self.use_cuda:
            targets = Variable(torch.LongTensor(targets)).cuda()
        else:
            targets = Variable(torch.LongTensor(targets))
            
        loss = F.cross_entropy(scores, targets)
        return loss

    def forward(self, input_batch, targets):
        nns  = self.preprocessing(input_batch)
        raw_output = self.createLocalNetworks(nns)
        loss = self._loss(raw_output, targets)
        return loss

In [21]:
model = LikeDislikeModel(2227, 62, 3, use_cuda = True)
if model.use_cuda:
    model.cuda()
output = model(data_set, targets)

/home/rauf/.conda/envs/my_root/lib/python2.7/site-packages/ipykernel_launcher.py:109: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [18]:
output

Variable containing:
 1.1211
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [55]:
torch.FloatTensor(2, 3, 5)


(0 ,.,.) = 
  2.2232e-21  4.5909e-41  4.6012e+09  3.0669e-41  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00

(1 ,.,.) = 
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  1.7937e-43  0.0000e+00
[torch.FloatTensor of size 2x3x5]